In [3]:
%matplotlib inline
# from ExKMC.Tree import Tree # import from cloned local library followed by installing manually
import sys
sys.path.append('../')
from ExKMC_M.ExKMC.Tree import Tree

from sklearn.datasets import make_blobs
import gdown
import pandas as pd
import copy
from sklearn.cluster import KMeans,DBSCAN
from utils import calc_cost, plot_kmeans, plot_tree_boundary,plot_confusion_matrix
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler, normalize
from utils import plot_confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import os

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

from sklearn.utils import check_random_state, check_X_y, _safe_indexing
from sklearn.metrics.pairwise import pairwise_distances, pairwise_distances_chunked
from sklearn.preprocessing import LabelEncoder
import functools
from sklearn.feature_selection import SelectKBest, mutual_info_regression,mutual_info_classif
from sklearn import metrics
from sklearn.decomposition import PCA,SparsePCA,MiniBatchSparsePCA


## data preprocessing

In [4]:
DF = pd.read_csv('/home/sfy/Documents/VScodeProject/Thesis/family.csv',index_col=False)
print(f'Data shape is {DF.shape}')

grouped = DF.groupby(['family'])
# get numbers of each group 
groupCount = grouped['family'].count()

Data shape is (20760, 698)


In [5]:
groupCount.sort_values(ascending=False)[:20]

family
Airpush         6652
FakeInst        2168
Mecor           1820
Youmi           1300
Fusob           1262
Kuguo           1197
Dowgin           862
BankBot          647
Jisut            558
DroidKungFu      546
RuMMS            402
Lotoor           329
Mseg             235
Boqx             215
Minimob          203
Triada           197
Kyview           175
SlemBunk         174
SimpleLocker     172
SmsKey           165
Name: family, dtype: int64

In [6]:
# groupCount
# seect the family with certain condition
# select by number of instances
# selected = groupCount[groupCount > 1000]

# select by numbers of the top 5
TOP = 11

selected = groupCount.sort_values(ascending=False)[1:TOP]

# get names
selectedNames = list(selected.index)
print(f"selected family are {selectedNames}")

# select trained
train = DF.loc[DF["family"].isin(selectedNames)]
train.reset_index(inplace=True, drop=True)
print(f"train dataset shape {train.shape}")


# encode into encoder
le = LabelEncoder()
label = le.fit_transform(train["family"])
# print(train.head(5))

# exclue name -> X_
train = train.loc[:, train.columns != "name"]
train = train.loc[:, train.columns != "family"]

# remove duplicates -> X
# parameter with first and last ? !!!! can be discussed
# X = train.drop_duplicates(keep='last')
X = train

# with X.index canbe reserved to original labels
Y = np.take(label, X.index)

print(f"Without duplications wi shape {X.shape}")
print(f"With duplications wi shape {train.shape}")

X.reset_index(inplace=True, drop=True)

# X.head(5)
# X.to_csv('testDup.csv',index=False)

selected family are ['FakeInst', 'Mecor', 'Youmi', 'Fusob', 'Kuguo', 'Dowgin', 'BankBot', 'Jisut', 'DroidKungFu', 'RuMMS']
train dataset shape (10762, 698)
Without duplications wi shape (10762, 696)
With duplications wi shape (10762, 696)


## evaluation

### fscore and accuracy

In [7]:
# from sklearn.feature_selection import VarianceThreshold,GenericUnivariateSelect,chi2
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1,random_state=0)
k = len(selectedNames)
X_train = X
X_train.shape

# success with variance
scaler = StandardScaler()
scaled_df = scaler.fit_transform(X_train)

# Converting the numpy array into a pandas DataFrame
X_train_ = pd.DataFrame(scaled_df)

# X_train_ = VarianceThreshold(.8*(1-.8)).fit_transform(X_train_)

kmeans = KMeans(k,random_state=0)

# kmeans_labels = kmeans.fit_predict(X_train_)
# kmeans_labels = kmeans.fit_transform(X_train_)
kmeans.fit(X_train_)

# tree = Tree(k=k,max_leaves=8 * k)
# tree_labels = tree.fit_predict(X_train_, kmeans)

# f_a_score(Y, kmeans_labels, tree_labels=tree_labels)
kmeans.inertia_

2285167.3047202765

In [15]:
# get jaccard
# get the sets index of each label kmeans predicted 
# get the sets index of true label 
# kmeans.labels_[5000:]

for i in range(TOP-1):
    # index of each true
    b = np.where(Y == i ) [0]
    similarity = list()
    for family in range(TOP-1):
        samples = np.where(kmeans.labels_ ==family)[0]
        correctly_classified = list(set(samples).intersection(set(b)))
        union = list(set(samples).union(set(b)))
        jind = len(correctly_classified) / float(len(union))
        similarity.append(jind)
    
    print(f'--Result of family label with {i}')

    print(similarity)


--Result of family label with 0
[0.09985315712187959, 0.0, 0.0, 0.0, 0.36882716049382713, 0.0, 0.0, 0.0, 0.0, 0.0]
--Result of family label with 1
[0.014652014652014652, 0.07728337236533958, 0.03552206673842842, 0.0, 0.0, 0.14374688589935228, 0.004634994206257242, 0.01524390243902439, 0.0, 0.0]
--Result of family label with 2
[0.052972195589645256, 0.1324200913242009, 0.0, 0.0, 0.0, 0.02223816355810617, 0.0, 0.0, 0.0, 0.0]
--Result of family label with 3
[0.5635560176761113, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
--Result of family label with 4
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.9984177215189873, 0.0]
--Result of family label with 5
[0.14415584415584415, 0.0010030090270812437, 0.0, 0.0, 0.0, 0.0002333177788147457, 0.0, 0.0, 0.0, 0.0]
--Result of family label with 6
[0.0017874875868917578, 0.22273781902552203, 0.051053484602917344, 0.0, 0.0, 0.1474493361285814, 0.0008326394671107411, 0.008314436885865457, 0.0, 0.0]
--Result of family label with 7
[0.0, 0.0, 0.0, 0.0, 0.0, 

true is 0 and label is 4
0 -4 0.36
3 -0 0.56
4 -8 0.99
7 -5 0.48
9 -1 0.25


In [17]:
test_X = X_train_
test_X['label'] = Y

test_X = test_X.drop(test_X[(test_X['label'] == 2)| (test_X['label'] ==3) | (test_X['label'] ==5) | (test_X['label'] ==6) | (test_X['label'] ==7)|(test_X['label'] ==8)].index)
test_X.drop('label', inplace=True, axis=1)
len(test_X)

test_Y = Y
# or can be deleted as index of X
test_Y = np.delete(test_Y,np.where((test_Y == 2 )|(test_Y == 3) |(test_Y == 5)|(test_Y == 6)|(test_Y == 7)|(test_Y == 8)))

#result
# intra and inter distance of kmenas is [16.55631207 15.89690893 -0.03070406]
len(test_Y)


In [31]:
index_X = test_X.index
predict = kmeans.labels_
len(predict)

# basePath = '/home/sfy/Documents/VScodeProject/Thesis'
basePath = "/home/sfy/Documents/VScodeProject/Thesis/result/"

# write distra.rsf true label
with open(basePath + os.sep + "distra.rsf", "w") as f:
    for line in range(len(index_X)):
        f.write("contain " + str(test_Y[line]) + " " + str(index_X[line]) + "\n")

# write distrb.rsf clustering result
with open(basePath + os.sep + "distrb.rsf", "w") as f:
    for line in range(len(index_X)):
        index = index_X[line]
        f.write("contain " + str(predict[index]) + " " + str(index) + "\n")

# java -cp /home/sfy/Documents/VScodeProject/Thesis/algorithms  mojo.MoJo /home/sfy/Documents/VScodeProject/Thesis/result/distra.rsf /home/sfy/Documents/VScodeProject/Thesis/result/distrb.rsf  -fm

## predict of dataset from 0-10
treue - predict
0-6 0.44
1-0 0.63
4-2 0.37
9-5 0.28

### result of duplication of 1-11 datasts
--Result of family label with 0 -4 0.36
[0.09985315712187959, 0.0, 0.0, 0.0, 0.36882716049382713, 0.0, 0.0, 0.0, 0.0, 0.0]
--Result of family label with 1 -5 0.13
[0.014652014652014652, 0.07728337236533958, 0.03552206673842842, 0.0, 0.0, 0.14374688589935228, 0.004634994206257242, 0.01524390243902439, 0.0, 0.0]
--Result of family label with 2 -1 0.13
[0.052972195589645256, 0.1324200913242009, 0.0, 0.0, 0.0, 0.02223816355810617, 0.0, 0.0, 0.0, 0.0]
--Result of family label with 3 -0 0.56
[0.5635560176761113, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
--Result of family label with 4 -8
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.9984177215189873, 0.0]
--Result of family label with 5 -0
[0.14415584415584415, 0.0010030090270812437, 0.0, 0.0, 0.0, 0.0002333177788147457, 0.0, 0.0, 0.0, 0.0]
--Result of family label with 6 -1
[0.0017874875868917578, 0.22273781902552203, 0.051053484602917344, 0.0, 0.0, 0.1474493361285814, 0.0008326394671107411, 0.008314436885865457, 0.0, 0.0]
--Result of family label with 7 -5
[0.0, 0.0, 0.0, 0.0, 0.0, 0.48806650576562083, 0.0, 0.0, 0.0, 0.0]
--Result of family label with 8 - 0
[0.10449701065765532, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
--Result of family label with 9 -1
[0.0031183005262132137, 0.2565396971087655, 0.0028653295128939827, 0.0007692307692307692, 0.000649772579597141, 0.13675406871609402, 0.0, 0.083710407239819, 0.00078064012490242, 0.0007692307692307692] 

result of duplication of all features


* no dupli

1. 0-2 61%;1-5 0.01; 2-5 0.24; 3-1 0.07; 4-5; 5-5; 7-5 0.25; 9-0.223
2. data imblanced?

* with duplication

--Result of family label with 0 - 6
[0.0002830455703368242, 0.35255452644158947, 0.0014452027298273785, 0.0001503307276007216, 0.0003003003003003003, 0.0063083522583901085, 0.44898612593383136, 0.00015026296018031557, 0.0001503307276007216, 0.0014814814814814814]
--Result of family label with 1 - 0
[0.6353846153846154, 0.0, 0.03753609239653513, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
--Result of family label with 2  - 6
[0.0, 0.0027649769585253456, 0.01750913520097442, 0.0, 0.0057670126874279125, 0.07051909892262488, 0.07689116304798783, 0.0, 0.0, 0.035218783351120594]
--Result of family label with 3 - 5
[0.0, 0.001358695652173913, 0.0009432479169941833, 0.0, 0.0, 0.08974358974358974, 0.05388630272252786, 0.0, 0.0, 0.0]
--Result of family label with 4 - 2
[0.0, 0.0, 0.37244459714825634, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
--Result of family label with 5 -2
[0.0, 0.000273000273000273, 0.21659223634489866, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
--Result of family label with 6 - 6
[0.0, 0.0, 0.016411727214786488, 0.0, 0.0, 0.0010638297872340426, 0.06443812233285917, 0.0, 0.0, 0.0]
--Result of family label with 7 -5
[0.0, 0.0025069637883008357, 0.009348482669351359, 0.0, 0.0008291873963515755, 0.1824577861163227, 0.08990874932903918, 0.0, 0.0, 0.050724637681159424]
--Result of family label with 8 -2
[0.0, 0.0, 0.31266105480158046, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
--Result of family label with 9 - 5
[0.0005830903790087463, 0.005158837903882704, 0.0043723554301833565, 0.0, 0.0015290519877675841, 0.28690534575772436, 0.08681289640591966, 0.0023059185242121443, 0.0, 0.001422475106685633]

## evaluation


In [20]:
# calculate the scores with jaccard

def f_a_score(true_labels, kmenas_labels, tree_labels):
    from sklearn.metrics import f1_score, accuracy_score

    print(f"fsocre of kmenas is {f1_score(y_true=true_labels,y_pred=kmenas_labels,average='macro')}")
    print(f"fsocre of tree score is {f1_score(y_true=true_labels,y_pred=tree_labels,average='macro')}")

    print(
        f"accuracy of kmenas is {accuracy_score(y_true=true_labels,y_pred=kmenas_labels)}"
    )
    print(
        f"accuracy of tree score is {accuracy_score(y_true=true_labels,y_pred=tree_labels)}"
    )


fsocre of kmenas is 0.07602760060373061
fsocre of tree score is 0.07051666035172852
accuracy of kmenas is 0.26633794013473155
accuracy of tree score is 0.24001666782415446


In [111]:
#random result

fsocre of kmenas is 0.5051308711179422
fsocre of tree score is 0.5307182751006624
accuracy of kmenas is 0.46932641733524005
accuracy of tree score is 0.4745618925273287


In [62]:


def _silhouette_reduce(D_chunk, start, labels, label_freqs):
    """Accumulate silhouette statistics for vertical chunk of X.

    Parameters
    ----------
    D_chunk : array-like of shape (n_chunk_samples, n_samples)
        Precomputed distances for a chunk.
    start : int
        First index in the chunk.
    labels : array-like of shape (n_samples,)
        Corresponding cluster labels, encoded as {0, ..., n_clusters-1}.
    label_freqs : array-like
        Distribution of cluster labels in ``labels``.
    """
    # accumulate distances from each sample to each cluster
    clust_dists = np.zeros((len(D_chunk), len(label_freqs)), dtype=D_chunk.dtype)
    for i in range(len(D_chunk)):
        clust_dists[i] += np.bincount(
            labels, weights=D_chunk[i], minlength=len(label_freqs)
        )

    # intra_index selects intra-cluster distances within clust_dists
    intra_index = (np.arange(len(D_chunk)), labels[start : start + len(D_chunk)])
    # intra_clust_dists are averaged over cluster size outside this function
    intra_clust_dists = clust_dists[intra_index]
    # of the remaining distances we normalise and extract the minimum
    clust_dists[intra_index] = np.inf
    clust_dists /= label_freqs
    inter_clust_dists = clust_dists.min(axis=1)
    return intra_clust_dists, inter_clust_dists


def silhouette_score(
    X, labels, *, metric="euclidean", sample_size=None, random_state=None, **kwds
):
    if sample_size is not None:
        X, labels = check_X_y(X, labels, accept_sparse=["csc", "csr"])
        random_state = check_random_state(random_state)
        indices = random_state.permutation(X.shape[0])[:sample_size]
        if metric == "precomputed":
            X, labels = X[indices].T[indices].T, labels[indices]
        else:
            X, labels = X[indices], labels[indices]
    else:
        scores = silhouette_samples(X, labels, metric=metric, **kwds)
    return np.mean(scores,axis=1)


def silhouette_samples(X, labels, *, metric="euclidean", **kwds):
    """Compute the Silhouette Coefficient for each sample.

    The Silhouette Coefficient is a measure of how well samples are clustered
    with samples that are similar to themselves. Clustering models with a high
    Silhouette Coefficient are said to be dense, where samples in the same
    cluster are similar to each other, and well separated, where samples in
    different clusters are not very similar to each other.

    The Silhouette Coefficient is calculated using the mean intra-cluster
    distance (``a``) and the mean nearest-cluster distance (``b``) for each
    sample.  The Silhouette Coefficient for a sample is ``(b - a) / max(a,
    b)``.
    Note that Silhouette Coefficient is only defined if number of labels
    is 2 ``<= n_labels <= n_samples - 1``.

    This function returns the Silhouette Coefficient for each sample.

    The best value is 1 and the worst value is -1. Values near 0 indicate
    overlapping clusters.
    """
    X, labels = check_X_y(X, labels, accept_sparse=["csc", "csr"])

    # Check for non-zero diagonal entries in precomputed distance matrix
    if metric == "precomputed":
        error_msg = ValueError(
            "The precomputed distance matrix contains non-zero "
            "elements on the diagonal. Use np.fill_diagonal(X, 0)."
        )
        if X.dtype.kind == "f":
            atol = np.finfo(X.dtype).eps * 100
            if np.any(np.abs(np.diagonal(X)) > atol):
                raise ValueError(error_msg)
        elif np.any(np.diagonal(X) != 0):  # integral dtype
            raise ValueError(error_msg)

    le = LabelEncoder()
    labels = le.fit_transform(labels)
    n_samples = len(labels)
    label_freqs = np.bincount(labels)

    kwds["metric"] = metric
    reduce_func = functools.partial(
        _silhouette_reduce, labels=labels, label_freqs=label_freqs
    )
    results = zip(*pairwise_distances_chunked(X, reduce_func=reduce_func, **kwds))
    intra_clust_dists, inter_clust_dists = results
    # intra:35655.59,inter:1.559
    intra_clust_dists = np.concatenate(intra_clust_dists)
    inter_clust_dists = np.concatenate(inter_clust_dists)
# unknow operatino of denom # after denom the dists become 1.49 before is 30234
    denom = (label_freqs - 1).take(labels, mode="clip")
    with np.errstate(divide="ignore", invalid="ignore"):
        intra_clust_dists /= denom

    sil_samples = inter_clust_dists - intra_clust_dists
    with np.errstate(divide="ignore", invalid="ignore"):
        sil_samples /= np.maximum(intra_clust_dists, inter_clust_dists)
    # nan values are for clusters of size 1, and should be 0
    return [intra_clust_dists,inter_clust_dists,np.nan_to_num(sil_samples)]


print(f'intra and inter distance of kmenas is {silhouette_score(X=X_train_, labels=Y)}')
# print(f'intra and inter distance of tree score is {silhouette_score(X=X_train, labels=tree_labels)}')


intra and inter distance of kmenas is [12.09037874 12.29912063  0.1893164 ]


In [70]:


metrics.v_measure_score( Y,kmeans.labels_)

0.5755774816415945

In [71]:
res = metrics.adjusted_rand_score(Y, kmeans.labels_)
res

0.4107813666326867

In [ ]:
# cost
# kmeas cost in paper: The k-means cost is the sum of squared distances of each point to the mean of points associated with the cluster.
# kmenas cost in sklearn:Opposite of the value of X on the K-means objective.
# surrogate cost:The k-means surrogate cost is the sum of squared distances of each point to the closest center of the kmeans given (or trained) in the fit method.k-means surrogate cost > k-means cost, as k-means cost is computed with respect to the optimal centers.

kmeas_cost = tree.score(X_train)
surrogate_score = tree.surrogate_score(X_train)
print(
    f"kmeans_cost is {kmeas_cost} \nsurrogate_score is {surrogate_score}\nkmenas cost in surrogate is {kmeans.score(X_train)}"
)

#test with distancs munally
# inter-distance
from scipy.spatial.distance import euclidean

dst = euclidean(kmeans.cluster_centers_[0], kmeans.cluster_centers_[1])
print(f"inter distance from kmeans:{dst}")

tree_centers = tree.get_centers()
tree_dst = euclidean(tree_centers[0], tree_centers[1])
print(f"inter distance from tree:{tree_dst}")

In [ ]:


# true = Y
# true = np.delete(true,np.where((true == 1 )|(true == 2) |(true == 5)|(true == 6)|(true == 8)))
# predict = kmeans.labels_
# predict = np.delete(predict,np.where((predict == 2 )|(predict == 3) |(predict == 6)|(predict == 9)|(predict == 7)))

# predict[predict==0] = 3
# predict[predict==8] = -4
# predict[predict==5] = 7
# predict[predict==1] = 9
# predict[predict==4] = 0
# predict[predict==-4] = 4

# #result
# # intra and inter distance of kmenas is [16.55631207 15.89690893 -0.03070406]
# len(true)

